# Using Upstage Document OCR on SageMaker Jumpstart


Upstage Document OCR (Optical Character Recognition) is designed to efficiently detect and recognize text from a wide range of document images, ensuring high accuracy and versatility across various languages and image qualities.

This sample notebook shows you how to deploy [Upstage Document OCR](https://aws.amazon.com/marketplace/pp/prodview-anvrh24vv3yiw) using Amazon SageMaker.

## Pre-requisites:

1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
2. Ensure that IAM role used has **AmazonSageMakerFullAccess**
3. To deploy this ML model successfully, ensure that:
   1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used:
      1. **aws-marketplace:ViewSubscriptions**
      2. **aws-marketplace:Unsubscribe**
      3. **aws-marketplace:Subscribe**

## Contents:

1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Prepare input payload](#B.-Prepare-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
3. [Clean-up](#4.-Clean-up)
   1. [Delete the endpoint](#A.-Delete-the-endpoint)
   2. [Delete the model](#B.-Delete-the-model)

## Usage instructions

You can run this notebook one cell at a time (By using Shift+Enter for running a cell).


## 1. Subscribe to the model package


To subscribe to the model package:

1. Open [Upstage Document OCR](https://aws.amazon.com/marketplace/pp/prodview-anvrh24vv3yiw) model package listing page.
2. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
3. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms.
4. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.


In [1]:
%pip -q install sagemaker requests_toolbelt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import boto3
import sagemaker
from sagemaker import ModelPackage, get_execution_role

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[02/05/25 00:36:08] INFO     Found credentials from IAM Role:                                   ]8;id=102902;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=632210;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
role = get_execution_role()
sagemaker_session = sagemaker.Session()

sagemaker_runtime = boto3.client("sagemaker-runtime")

[02/05/25 00:36:12] INFO     Found credentials from IAM Role:                                   ]8;id=146440;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=30347;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

[02/05/25 00:36:13] INFO     Found credentials from IAM Role:                                   ]8;id=575234;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=759309;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     Found credentials from IAM Role:                                   ]8;id=295931;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=110552;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [4]:
model_package_name = "upstage-document-ocr-2-2-1-r2-a9daefb5d35b3356b5a968b6fa8e8b9a"

# Mapping for Model Packages
model_package_map = {
    "us-east-1": f"arn:aws:sagemaker:us-east-1:865070037744:model-package/{model_package_name}",
    "us-east-2": f"arn:aws:sagemaker:us-east-2:057799348421:model-package/{model_package_name}",
    "us-west-1": f"arn:aws:sagemaker:us-west-1:382657785993:model-package/{model_package_name}",
    "us-west-2": f"arn:aws:sagemaker:us-west-2:594846645681:model-package/{model_package_name}",
    "ca-central-1": f"arn:aws:sagemaker:ca-central-1:470592106596:model-package/{model_package_name}",
    "eu-central-1": f"arn:aws:sagemaker:eu-central-1:446921602837:model-package/{model_package_name}",
    "eu-west-1": f"arn:aws:sagemaker:eu-west-1:985815980388:model-package/{model_package_name}",
    "eu-west-2": f"arn:aws:sagemaker:eu-west-2:856760150666:model-package/{model_package_name}",
    "eu-west-3": f"arn:aws:sagemaker:eu-west-3:843114510376:model-package/{model_package_name}",
    "eu-north-1": f"arn:aws:sagemaker:eu-north-1:136758871317:model-package/{model_package_name}",
    "ap-southeast-1": f"arn:aws:sagemaker:ap-southeast-1:192199979996:model-package/{model_package_name}",
    "ap-southeast-2": f"arn:aws:sagemaker:ap-southeast-2:666831318237:model-package/{model_package_name}",
    "ap-northeast-2": f"arn:aws:sagemaker:ap-northeast-2:745090734665:model-package/{model_package_name}",
    "ap-northeast-1": f"arn:aws:sagemaker:ap-northeast-1:977537786026:model-package/{model_package_name}",
    "ap-south-1": f"arn:aws:sagemaker:ap-south-1:077584701553:model-package/{model_package_name}",
    "sa-east-1": f"arn:aws:sagemaker:sa-east-1:270155090741:model-package/{model_package_name}",
}

region = sagemaker_session.boto_region_name
if region not in model_package_map.keys():
    raise Exception(f"Current boto3 session region {region} is not supported.")

model_package_arn = model_package_map[region]

print(f"Model Package: '{model_package_arn}'")

Model Package: 'arn:aws:sagemaker:ap-northeast-2:745090734665:model-package/upstage-document-ocr-2-2-1-r2-a9daefb5d35b3356b5a968b6fa8e8b9a'


## 2. Create an endpoint and perform real-time inference


If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).


In [10]:
model_name = "Upstage-Document-OCR"

real_time_inference_instance_type = "ml.g5.xlarge"

### A. Create an endpoint


In [11]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

endpoint_name = sagemaker.utils.name_from_base(model_name)
print(f"endpoint name: '{endpoint_name}'")

endpoint name: 'Upstage-Document-OCR-2025-02-05-00-39-09-001'


In [12]:
# Deploy the model
model.deploy(1, real_time_inference_instance_type, endpoint_name=endpoint_name)

[02/05/25 00:39:09] INFO     Creating model with name:                                              ]8;id=130650;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=481832;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\
                             upstage-document-ocr-2-2-1-r2-a9daefb5d-2025-02-05-00-39-09-189                       

                    INFO     Creating endpoint-config with name                                     ]8;id=861887;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=155634;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#5889\5889]8;;\
                             Upstage-Document-OCR-2025-02-05-00-39-09-001                                          

                    INFO     Creating endpoint with name                                            ]8;id=787009;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=945179;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4711\4711]8;;\
                             Upstage-Document-OCR-2025-02-05-00-39-09-001                                          

-----------------!

Once endpoint has been created, you would be able to perform real-time inference.


### B. Prepare input payload


We support jpg, png, bmp, heic, pdf, tiff formats.


### C. Perform real-time inference


In [13]:
import boto3
import json
from requests_toolbelt import MultipartEncoder

runtime_sm_client = boto3.client("runtime.sagemaker")

# Prepare multipart form data
m = MultipartEncoder(
    fields={
        "document": ("sample_1.png", open("sample_1.png", "rb"), "image/png"),
        # 'document': ('solar.pdf', open('solar.pdf', 'rb'), 'application/pdf'), # for PDF files
        "model": "ocr",
        "use_patchify": "false",  # Enable this flag when you need more accurate inference for images larger than 2560.
    }
)

# Get the raw bytes of the multipart form data
body = m.to_string()

response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType=m.content_type,  # This will be 'multipart/form-data; boundary=...'
    Body=body,
)

result = response["Body"].read()

# Print the result
print("output:", json.loads(result))

output: {'apiVersion': '1.1', 'confidence': 0.9919, 'metadata': {'pages': [{'height': 125, 'page': 1, 'width': 384}]}, 'mimeType': 'multipart/form-data', 'modelVersion': 'ocr-2.2.1', 'numBilledPages': 1, 'pages': [{'confidence': 0.9919, 'height': 125, 'id': 0, 'text': 'Print the words \nhello, world', 'width': 384, 'words': [{'boundingBox': {'vertices': [{'x': 31, 'y': 25}, {'x': 110, 'y': 25}, {'x': 110, 'y': 52}, {'x': 31, 'y': 52}]}, 'confidence': 0.991, 'id': 0, 'text': 'Print'}, {'boundingBox': {'vertices': [{'x': 118, 'y': 21}, {'x': 170, 'y': 27}, {'x': 167, 'y': 55}, {'x': 115, 'y': 50}]}, 'confidence': 0.999, 'id': 1, 'text': 'the'}, {'boundingBox': {'vertices': [{'x': 178, 'y': 26}, {'x': 272, 'y': 26}, {'x': 272, 'y': 53}, {'x': 178, 'y': 53}]}, 'confidence': 0.9892, 'id': 2, 'text': 'words'}, {'boundingBox': {'vertices': [{'x': 102, 'y': 65}, {'x': 230, 'y': 71}, {'x': 228, 'y': 101}, {'x': 101, 'y': 95}]}, 'confidence': 0.9927, 'id': 3, 'text': 'hello,'}, {'boundingBox': {

## 3. Clean-up


### A. Delete the endpoint


Now that you have successfully performed a real-time inference, you can delete the endpoint and avoid being charged.


In [14]:
model.sagemaker_session.delete_endpoint(endpoint_name)
model.sagemaker_session.delete_endpoint_config(endpoint_name)

[02/05/25 00:48:13] INFO     Deleting endpoint with name:                                           ]8;id=697793;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=980672;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4855\4855]8;;\
                             Upstage-Document-OCR-2025-02-05-00-39-09-001                                          

                    INFO     Deleting endpoint configuration with name:                             ]8;id=372625;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=707229;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4865\4865]8;;\
                             Upstage-Document-OCR-2025-02-05-00-39-09-001                                          

### B. Delete the model


In [15]:
model.delete_model()

                    INFO     Deleting model with name:                                              ]8;id=737903;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=493484;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#5226\5226]8;;\
                             upstage-document-ocr-2-2-1-r2-a9daefb5d-2025-02-05-00-39-09-189                       